# Scores

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tok_ids = tokenizer.encode("Hello, my name is", return_tensors="pt")
print(tok_ids)
print(tok_ids.shape)

tensor([[15496,    11,   616,  1438,   318]])
torch.Size([1, 5])


/workspaces/distributed-speculative-inference/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [45]:
import torch

model = AutoModelForCausalLM.from_pretrained(model_name)
outputs = model.generate(
    input_ids=tok_ids,
    attention_mask=torch.ones_like(tok_ids),
    max_new_tokens=10,
    do_sample=False,
    use_cache=False,
    return_dict_in_generate=True,
    output_logits=False,
    output_scores=True,
    output_hidden_states=False,
    output_attentions=False,
)
print(outputs)
# print(outputs.shape)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GenerateDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  1438,   318,  1757,    13,   314,  1101,   257,
          6260,    11,   290,   314,  1101]]), scores=(tensor([[-68.4289, -69.0397, -73.2529,  ..., -76.9531, -77.1063, -70.0094]]), tensor([[-65.0738, -66.4413, -71.7042,  ..., -79.9795, -77.5036, -68.6040]]), tensor([[-130.1886, -129.3490, -131.3795,  ..., -141.4483, -142.7363,
         -124.0823]]), tensor([[-131.9187, -130.0697, -135.7836,  ..., -141.5104, -134.1392,
         -133.0103]]), tensor([[-112.3013, -111.0005, -114.8719,  ..., -119.3572, -119.8524,
         -112.7859]]), tensor([[-108.1326, -106.7145, -110.4225,  ..., -115.2540, -114.6338,
         -107.9902]]), tensor([[ -95.4683,  -96.0403, -103.7461,  ..., -108.3876, -107.8743,
          -99.9795]]), tensor([[-83.7534, -83.9493, -85.5163,  ..., -88.0616, -89.6183, -82.4906]]), tensor([[-110.4278, -109.9855, -113.4881,  ..., -116.3650, -116.2361,
         -110.5043]]), tensor([[-144.8037, -143.3270, -147.85

In [14]:
# print all the fields of the `outputs` object
print(outputs.keys())

odict_keys(['sequences', 'scores'])


In [22]:
print(outputs.sequences)
print(tokenizer.batch_decode(outputs.sequences))

tensor([[15496,    11,   616,  1438,   318,  1757,    13,   314,  1101,   257,
          6260,    11,   290,   314,  1101]])
["Hello, my name is John. I'm a writer, and I'm"]


In [34]:
print(type(outputs.scores))
print(outputs.scores)

<class 'tuple'>
(tensor([[-68.4289, -69.0397, -73.2529,  ..., -76.9531, -77.1063, -70.0094]]), tensor([[-65.0738, -66.4413, -71.7042,  ..., -79.9795, -77.5036, -68.6040]]), tensor([[-130.1886, -129.3490, -131.3795,  ..., -141.4483, -142.7363,
         -124.0823]]), tensor([[-131.9187, -130.0697, -135.7836,  ..., -141.5104, -134.1392,
         -133.0103]]), tensor([[-112.3013, -111.0005, -114.8719,  ..., -119.3572, -119.8524,
         -112.7859]]), tensor([[-108.1326, -106.7145, -110.4225,  ..., -115.2540, -114.6338,
         -107.9902]]), tensor([[ -95.4683,  -96.0403, -103.7461,  ..., -108.3876, -107.8743,
          -99.9795]]), tensor([[-83.7534, -83.9493, -85.5163,  ..., -88.0616, -89.6183, -82.4906]]), tensor([[-110.4278, -109.9855, -113.4881,  ..., -116.3650, -116.2361,
         -110.5043]]), tensor([[-144.8037, -143.3270, -147.8548,  ..., -154.3877, -147.2710,
         -145.4144]]))


In [46]:
print(len(outputs.scores))
print(outputs.scores[0].shape)
# Stack the scores as follows.
# Instead of the existing shape of the a score (batch=1, vocab), the first dimension is
# the sequence and the second is the vocab (num_of_scores, vocab). Since the batch size
# is always 1, we can omit the batch dimension. Note that we might get `num_of_scores=1`
torch.stack(outputs.scores, dim=0).squeeze(1).shape

10
torch.Size([1, 50257])


torch.Size([10, 50257])

In [26]:
argmax = [s.argmax() for s in outputs.scores]
print(argmax)
print(tokenizer.batch_decode(argmax))

[tensor(1757), tensor(13), tensor(314), tensor(1101), tensor(257), tensor(6260), tensor(11), tensor(290), tensor(314), tensor(1101)]
[' John', '.', ' I', "'m", ' a', ' writer', ',', ' and', ' I', "'m"]


# Preemption

Proof-of-concept that we can preempt a long-running call to `transformers`.